# Tesla subsidies - Good Jobs First

#### Load Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import altair as alt
import altair_stiles as altstiles
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g

In [3]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('grid')

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.set_option("display.float_format", lambda x: "%.f" % x)

#### Google Sheets config

In [5]:
scope = ["https://spreadsheets.google.com/feeds"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    "credentials.json", scope
)
gc = gspread.authorize(credentials)
spreadsheet_key = "1vlLpqst3gASjVFJD1uGYbLahG4Nrvce5Mz17me9EOW4"
book = gc.open_by_key(spreadsheet_key)

---

In [6]:
url = "https://subsidytracker.goodjobsfirst.org/prog.php?parent=tesla-inc&detail=export_csv"

In [7]:
src = pd.read_csv(
    url,
    dtype={
        "NAICS Industry Code": str,
        "Zip": str,
        "CFDA Program Number": str,
        "Year": str,
    },
)

In [8]:
src.columns = src.columns.str.lower().str.replace(" ", "_", regex=False)

In [9]:
src.subsidy_value = (
    src.subsidy_value.str.replace(",", "", regex=False)
    .str.replace("$", "", regex=False)
    .str.replace("undisclosed", "0")
    .fillna("0")
    .astype(float)
)

In [10]:
src.subsidy_value_adjusted_for_megadeal = (
    src.subsidy_value_adjusted_for_megadeal.str.replace(",", "", regex=False)
    .str.replace("$", "", regex=False)
    .str.replace("undisclosed", "0")
    .fillna("0")
    .astype(float)
)

In [11]:
df = (
    src[
        [
            "parent_company",
            "company",
            "year",
            "subsidy_value",
            "subsidy_value_adjusted_for_megadeal",
            "type_of_subsidy",
            "program_name",
            "awarding_agency",
            "number_of_jobs_or_training_slots",
            "subsidy_source",
        ]
    ]
    .sort_values("subsidy_value", ascending=False)
    .copy()
)

In [27]:
df.head()

,parent_company,company,year,subsidy_value,subsidy_value_adjusted_for_megadeal,type_of_subsidy,program_name,awarding_agency,number_of_jobs_or_training_slots,subsidy_source
11,Tesla Inc.,Tesla Motors,2014,1287000000,212206028,MEGADEAL,multiple,multiple,6500,multiple
93,Tesla Inc.,SolarCity Corp.,2014,750000000,750000000,MEGADEAL,multiple,multiple,1500,state
107,Tesla Inc.,Tesla Motors,2015,725800000,725800000,tax credit/rebate,Sales and Use Tax Abatement,Nevada Governor's Office of Economic Development,6500,state
105,Tesla Inc.,Tesla Motors,2015,245856460,245856460,tax credit/rebate,Personal Property Tax Abatement,Nevada Governor's Office of Economic Development,6500,state
106,Tesla Inc.,Tesla Motors,2015,103137512,103137512,tax credit/rebate,Real Property Tax Abatement,Nevada Governor's Office of Economic Development,6500,state


---

#### Total value of subsidies?

In [22]:
total = (
    df.groupby(["parent_company"])
    .agg({"subsidy_value_adjusted_for_megadeal": sum})
    .reset_index()
)

#### Types

In [14]:
types = (
    df.groupby("type_of_subsidy")
    .agg({"subsidy_value_adjusted_for_megadeal": sum})
    .reset_index()
    .sort_values("subsidy_value_adjusted_for_megadeal", ascending=False)
)
types["share"] = (
    types["subsidy_value_adjusted_for_megadeal"]
    / types["subsidy_value_adjusted_for_megadeal"].sum()
) * 100
types.head(10)

,type_of_subsidy,subsidy_value_adjusted_for_megadeal,share
5,tax credit/rebate,1136657116,45
0,MEGADEAL,1026739691,41
1,federal grant,339597164,14
3,grant,2400000,0
6,training reimbursement,722282,0
2,federal loan or loan guarantee,0,0
4,property tax abatement,0,0


#### Companies

In [29]:
companies = (
    df.groupby(["company"])
    .agg({"subsidy_value_adjusted_for_megadeal": sum})
    .reset_index()
    .sort_values("subsidy_value_adjusted_for_megadeal", ascending=False)
)
companies["share"] = (
    companies["subsidy_value_adjusted_for_megadeal"]
    / companies["subsidy_value_adjusted_for_megadeal"].sum()
) * 100
companies = companies[companies["subsidy_value_adjusted_for_megadeal"] > 0]
companies

,company,subsidy_value_adjusted_for_megadeal,share
23,Tesla Motors,1316363655,53
15,SolarCity Corp.,750000000,30
12,"Sequoia Pacific Solar I, LLC",86804325,3
1,"Eiger Lease Co, LLC",71126247,3
22,Tesla,64533663,3
4,"Mound Solar Master Tenant V, LLC",58188350,2
28,"USB SolarCity Master Tenant IV, LLC",40393256,2
27,"USB SolarCity Master Tenant III, LLC",23900192,1
26,"USB SolarCity Master Tenant 2009-2010, LLC",18615160,1
24,"Tesla Motors, Inc.",15000000,1


#### Sources

In [28]:
sources = (
    df.groupby(["subsidy_source"])
    .agg({"subsidy_value_adjusted_for_megadeal": sum})
    .reset_index()
    .sort_values("subsidy_value_adjusted_for_megadeal", ascending=False)
)
sources["share"] = (
    sources["subsidy_value_adjusted_for_megadeal"]
    / sources["subsidy_value_adjusted_for_megadeal"].sum()
) * 100
sources = sources[sources["subsidy_value_adjusted_for_megadeal"] > 0]
sources

---

## Export

In [17]:
wks_name = "sources"
d2g.upload(sources, spreadsheet_key, wks_name, credentials=credentials, row_names=False)

<Worksheet 'sources' id:1478792667>

In [18]:
wks_name = "companies"
d2g.upload(
    companies, spreadsheet_key, wks_name, credentials=credentials, row_names=False
)

<Worksheet 'companies' id:489141158>

In [19]:
wks_name = "types"
d2g.upload(types, spreadsheet_key, wks_name, credentials=credentials, row_names=False)

<Worksheet 'types' id:576489948>

In [20]:
wks_name = "all_subsidies"
d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=False)

<Worksheet 'all_subsidies' id:0>

In [23]:
wks_name = "total"
d2g.upload(total, spreadsheet_key, wks_name, credentials=credentials, row_names=False)

<Worksheet 'total' id:1237727761>